# Machine Intelligence with Deep Learning
## Importance batching for improved training of neural networks
---

In [1]:
import timeit

In [2]:
from models.resnet import ResNet18
from utils.data_utils import DataLoader
from utils.logging_utils import *

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import os
import pandas as pd

from datetime import datetime
today = datetime.today().strftime('%Y%m%d')

In [3]:
SEEDS = [10, 42, 4] # don't change!
#STRATEGIES = ['freeze', 'shuffle', 'homogeneous', 'heterogeneous', 'max_k_loss', 'min_k_loss'] # can be changed
STRATEGIES = ['max_k_loss', 'min_k_loss']
# FUTURE: heterogeneous_max_k_loss, ...min_loss

In [4]:
### Training
def train(epoch, optimizer, criterion_fn, seed, dataloader, strategy, device):
    criterion = criterion_fn()
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(dataloader.yield_batches(strategy, \
                                                  random_state=seed, use_train=True, \
                                                  criterion=criterion_fn(reduction='none'),\
                                                  device=device, num_iterations=500)): # 500: 50,000 images / 100 batch size
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    train_acc = 100.*correct/total
    train_loss /= total
    return train_acc, train_loss

### Testing
def test(epoch, best_acc, criterion_fn, seed, dataloader):
    criterion = criterion_fn()
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader.yield_batches('shuffle', \
                                                                               random_state=seed, use_train=False)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Save checkpoint.
    test_acc = 100.*correct/total
    test_loss /= total
    if test_acc > best_acc:
        best_acc = test_acc
        net.save(best_acc, epoch, seed, strategy)
    return test_acc, test_loss

In [5]:
resume = False
given_date = '20191113' #only needed if resumed from checkpoint
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
### task: classification of the following classes
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

    
### hyperparameters
test_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
num_epochs = 100 #100 # number of epochs the model gets trained
learning_rate = 0.01
momentum = 0.9
batch_size = 100
weight_decay = 5e-4

print("Begin training.")
start = timeit.default_timer()
#Logging header
length_table = 90
log_separating_line(length_table)
log_header_line("Seeds: {}".format(SEEDS), length_table)
log_header_line("Strategies: {}".format(STRATEGIES), length_table)
log_header_line("-> Resulting number of iterations: {}".format(len(SEEDS) * len(STRATEGIES)), length_table)
log_header_line("Number of iterations: {}".format(num_epochs), length_table)
log_header_line("Learning rate: {}".format(learning_rate), length_table)
log_header_line("Resuming from checkpoint: {}".format(True if resume else False), length_table)
log_separating_line(length_table)

rows = []
for seed in SEEDS:
    for strategy in STRATEGIES:
                
        np.random.seed(seed)
        torch.manual_seed(seed)
        if device == 'cuda':
            torch.cuda.manual_seed_all(seed)
             
        ### Model
        net = ResNet18()
        net = net.to(device)
        criterion = nn.CrossEntropyLoss # no function!
        optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
        
        ### load the data
        # if needed, specify batch sizes and shuffle settings
        dataloader = DataLoader(batch_size=batch_size)
        dataloader.download_cifar()
        dataloader.set_model(net)
        print()
        log_separating_line(length_table)
        log_position_header(seed, strategy, length_table)
        log_separating_line(length_table)

        if resume:
            assert os.path.isdir('serialized'), 'Error: no serialized directory found!'
            ckpt = torch.load('./serialized/{}/{}_ckpt_{}.pth'.format(given_date, strategy, seed))
            test_acc, start_epoch, net = net.load(ckpt)

        for epoch in range(start_epoch, start_epoch+num_epochs):
            if strategy in ['max_k_loss', 'min_k_loss']:
                dataloader.initialize_weights(criterion(reduction='none'), device, seed=seed, 
                                              dump='./dump_logs/{}_{}_{}.txt'.format(strategy, seed, epoch))
            if strategy in ['heterogeneous_max_k_loss', 'heterogeneous_min_k_loss']:
                dataloader.initialize_weights_per_class(criterion(reduction='none'), device, seed=seed)

            train_acc, train_loss = train(epoch, optimizer, criterion, seed, dataloader, \
                                          strategy, device)
            test_acc, test_loss = test(epoch, test_acc, criterion, seed, dataloader)
            log_position_line(epoch + 1, num_epochs, train_acc, test_acc, train_loss, test_loss, length_table)
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': True,
                'strategy': strategy,
                'accuracy': train_acc,
                'loss': train_loss
            }
            rows.append(row)
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': False,
                'strategy': strategy,
                'accuracy': test_acc,
                'loss': test_loss
            }
            rows.append(row)
        log_separating_line(length_table)
            
stop = timeit.default_timer()
time_needed = stop - start
hrs = int(time_needed / 3600)
mins = int((time_needed / 60) % 60)
secs = int(time_needed % 60)
print()
print("Finished training. Time needed: {} hrs {} mins {} secs".format(hrs, mins, secs))

logging_df = pd.DataFrame(rows, columns=['epoch', 'seed', 'train', 'strategy', 'accuracy', 'loss'])   
training_logs_dir = 'evaluation_logs'
logging_df.to_csv('{}.txt'.format(os.path.join(training_logs_dir, today)), sep='\t', index=False)

Begin training.
+----------------------------------------------------------------------------------------+
| Seeds: [10, 42, 4]                                                                     |
| Strategies: ['max_k_loss', 'min_k_loss']                                               |
| -> Resulting number of iterations: 6                                                   |
| Number of iterations: 100                                                              |
| Learning rate: 0.01                                                                    |
| Resuming from checkpoint: False                                                        |
+----------------------------------------------------------------------------------------+
Files already downloaded and verified
Files already downloaded and verified

+----------------------------------------------------------------------------------------+
| Seed: 10      Strategy: max_k_loss                                                    

| [078/100]:    98.85           77.43           0.00044      0.01272                     |
| [079/100]:    98.33           78.32           0.00064      0.01155                     |
| [080/100]:    98.36           79.25           0.00062      0.01141                     |
| [081/100]:    98.37           78.71           0.00061      00.0118                     |
| [082/100]:    98.76           77.95           0.00047      0.01157                     |
| [083/100]:    98.21           078.5           0.00069      0.01161                     |
| [084/100]:    98.48           76.96           0.00056      0.01369                     |
| [085/100]:    98.31           76.75           0.00068      0.01208                     |
| [086/100]:    98.27           077.5           0.00067      0.01183                     |
| [087/100]:    98.49           076.6           0.00057      0.01392                     |
| [088/100]:    98.77           78.41           0.00047      0.01206                     |

| [063/100]:    100.0           010.0           00000.0      0.06829                     |
| [064/100]:    100.0           010.0           00000.0      0.06831                     |
| [065/100]:    100.0           010.0           00000.0      0.06828                     |
| [066/100]:    100.0           010.0           00000.0      0.06811                     |
| [067/100]:    100.0           010.0           00000.0      0.06792                     |
| [068/100]:    100.0           010.0           00000.0      0.06773                     |
| [069/100]:    100.0           010.0           00000.0      00.0675                     |
| [070/100]:    100.0           010.0           00000.0      0.06723                     |
| [071/100]:    100.0           010.0           00000.0      0.06694                     |
| [072/100]:    100.0           010.0           00000.0      0.06662                     |
| [073/100]:    100.0           010.0           00000.0      0.06629                     |

| [048/100]:    98.58           78.13           0.00054      00.0142                     |
| [049/100]:    96.66           75.28           0.00145      0.01623                     |
| [050/100]:    98.08           78.73           0.00076      0.01292                     |
| [051/100]:    098.2           75.86           0.00072      0.01722                     |
| [052/100]:    98.12           76.71           0.00077      00.0135                     |
| [053/100]:    98.26           78.15           0.00067      0.01266                     |
| [054/100]:    98.16           78.64           0.00071      00.0173                     |
| [055/100]:    98.77           76.07           0.00048      0.01654                     |
| [056/100]:    98.12           77.18           0.00074      0.01312                     |
| [057/100]:    98.25           78.34           0.00069      0.01186                     |
| [058/100]:    98.77           78.32           00.0005      0.01324                     |

| [033/100]:    100.0           010.0           00000.0      00.0722                     |
| [034/100]:    100.0           010.0           00000.0      0.07758                     |
| [035/100]:    100.0           010.0           00000.0      0.07243                     |
| [036/100]:    100.0           010.0           00000.0      000.072                     |
| [037/100]:    100.0           010.0           00000.0      0.06834                     |
| [038/100]:    100.0           010.0           00000.0      0.06596                     |
| [039/100]:    100.0           010.0           00000.0      0.06406                     |
| [040/100]:    100.0           010.0           00000.0      0.06254                     |
| [041/100]:    100.0           010.0           00000.0      0.05881                     |
| [042/100]:    100.0           010.0           00000.0      00.0685                     |
| [043/100]:    100.0           010.0           00000.0      0.07026                     |

| [018/100]:    92.74           74.61           00.0023      0.01234                     |
| [019/100]:    94.14           077.2           0.00198      0.01144                     |
| [020/100]:    95.85           76.74           0.00142      0.01236                     |
| [021/100]:    96.66           77.43           0.00121      0.01231                     |
| [022/100]:    97.18           76.79           0.00098      0.01311                     |
| [023/100]:    97.28           77.03           0.00099      0.01313                     |
| [024/100]:    97.89           76.97           0.00077      0.01357                     |
| [025/100]:    98.56           76.64           0.00055      00.0137                     |
| [026/100]:    98.61           77.05           0.00054      0.01421                     |
| [027/100]:    98.33           76.64           0.00063      0.01416                     |
| [028/100]:    98.36           77.38           00.0006      0.01369                     |

| [003/100]:    100.0           010.0           00000.0      0.74383                     |
| [004/100]:    100.0           010.0           00000.0      0.70755                     |
| [005/100]:    100.0           010.0           00000.0      0.67303                     |
| [006/100]:    100.0           010.0           00000.0      00.6402                     |
| [007/100]:    100.0           010.0           00000.0      0.60898                     |
| [008/100]:    100.0           010.0           00000.0      0.57927                     |
| [009/100]:    100.0           010.0           00000.0      0.55101                     |
| [010/100]:    100.0           010.0           00000.0      0.52414                     |
| [011/100]:    100.0           010.0           00000.0      0.49857                     |
| [012/100]:    100.0           010.0           00000.0      0.47425                     |
| [013/100]:    100.0           010.0           00000.0      0.45111                     |

| [094/100]:    100.0           010.0           00000.0      0.04862                     |
| [095/100]:    100.0           010.0           00000.0      0.04864                     |
| [096/100]:    100.0           010.0           00000.0      0.04868                     |
| [097/100]:    100.0           010.0           00000.0      0.04873                     |
| [098/100]:    100.0           010.0           00000.0      0.04878                     |
| [099/100]:    100.0           010.0           00000.0      0.04886                     |
| [100/100]:    100.0           010.0           00000.0      0.04896                     |
+----------------------------------------------------------------------------------------+

Finished training. Time needed: 42 hrs 10 mins 26 secs
